# Example: Binary classification
--------------------------------

This example shows how to use ATOM to solve a binary classification problem. Additonnaly, we'll perform a variety of data cleaning steps to prepare the data for modeling.

The data used is a variation on the [Australian weather dataset](https://www.kaggle.com/jsphyg/weather-dataset-rattle-package) from Kaggle. You can download it from [here](https://github.com/tvdboom/ATOM/blob/master/examples/datasets/weatherAUS.csv). The goal of this dataset is to predict whether or not it will rain tomorrow training a binary classifier on target `RainTomorrow`.

## Load the data

In [13]:
# Import packages
import pandas as pd
from atom import ATOMClassifier

In [14]:
# Load data
X = pd.read_csv("./datasets/weatherAUS.csv")

# Let's have a look
X.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,MelbourneAirport,18.0,26.9,21.4,7.0,8.9,SSE,41.0,W,SSE,...,95.0,54.0,1019.5,1017.0,8.0,5.0,18.5,26.0,Yes,0
1,Adelaide,17.2,23.4,0.0,NaN,NaN,S,41.0,S,WSW,...,59.0,36.0,1015.7,1015.7,NaN,NaN,17.7,21.9,No,0
2,Cairns,18.6,24.6,7.4,3.0,6.1,SSE,54.0,SSE,SE,...,78.0,57.0,1018.7,1016.6,3.0,3.0,20.8,24.1,Yes,0
3,Portland,13.6,16.8,4.2,1.2,0.0,ESE,39.0,ESE,ESE,...,76.0,74.0,1021.4,1020.5,7.0,8.0,15.6,16.0,Yes,1
4,Walpole,16.4,19.9,0.0,NaN,NaN,SE,44.0,SE,SE,...,78.0,70.0,1019.4,1018.9,NaN,NaN,17.4,18.1,No,0


## Run the pipeline

In [15]:
# Call atom using only 5% of the complete dataset (for explanatory purposes)
atom = ATOMClassifier(X, y="RainTomorrow", n_rows=0.05, n_jobs=8, verbose=2)

<< ================== ATOM ================== >>

Configuration ==================== >>
Algorithm task: Binary classification.
Parallel processing with 8 cores.

Dataset stats ==================== >>
Shape: (7109, 22)
Train set size: 5688
Test set size: 1421
-------------------------------------
Memory: 1.25 MB
Scaled: False
Missing values: 15772 (10.1%)
Categorical features: 5 (23.8%)



In [16]:
# Impute missing values
atom.impute(strat_num="median", strat_cat="drop", max_nan_rows=0.8)

Fitting Imputer...
Imputing missing values...
 --> Dropping 3 samples for containing more than 16 missing values.
 --> Dropping 896 samples for containing missing values in categorical columns.
 --> Imputing 10 missing values with median (12.2) in column MinTemp.
 --> Imputing 7 missing values with median (22.9) in column MaxTemp.
 --> Imputing 2560 missing values with median (4.8) in column Evaporation.
 --> Imputing 2841 missing values with median (8.4) in column Sunshine.
 --> Imputing 45 missing values with median (70.0) in column Humidity9am.
 --> Imputing 76 missing values with median (52.0) in column Humidity3pm.
 --> Imputing 470 missing values with median (1017.4) in column Pressure9am.
 --> Imputing 468 missing values with median (1015.0) in column Pressure3pm.
 --> Imputing 2346 missing values with median (5.0) in column Cloud9am.
 --> Imputing 2459 missing values with median (5.0) in column Cloud3pm.
 --> Imputing 16 missing values with median (16.95) in column Temp9am.
 --

In [17]:
# Encode the categorical features
atom.encode(strategy="Target", max_onehot=10, infrequent_to_value=0.04)

Fitting Encoder...
Encoding categorical columns...
 --> Target-encoding feature Location. Contains 1 classes.
   --> Handling 6210 unknown classes.
 --> Target-encoding feature WindGustDir. Contains 16 classes.
 --> Target-encoding feature WindDir9am. Contains 16 classes.
 --> Target-encoding feature WindDir3pm. Contains 16 classes.
 --> Ordinal-encoding feature RainToday. Contains 2 classes.


In [18]:
# Train an Extra-Trees and a Random Forest model
atom.run(models=["ET", "RF"], metric="f1", n_bootstrap=5)


Training ========================= >>
Models: ET, RF
Metric: f1


Results for ExtraTrees:
Fit ---------------------------------------------
Train evaluation --> f1: 1.0
Test evaluation --> f1: 0.5349
Time elapsed: 0.273s
Bootstrap ---------------------------------------
Evaluation --> f1: 0.5655 ± 0.0068
Time elapsed: 0.902s
-------------------------------------------------
Time: 1.175s


Results for RandomForest:
Fit ---------------------------------------------
Train evaluation --> f1: 1.0
Test evaluation --> f1: 0.5714
Time elapsed: 0.261s
Bootstrap ---------------------------------------
Evaluation --> f1: 0.5718 ± 0.0131
Time elapsed: 1.012s
-------------------------------------------------
Time: 1.273s


Final results ==================== >>
Total time: 2.458s
-------------------------------------
ExtraTrees   --> f1: 0.5655 ± 0.0068 ~
RandomForest --> f1: 0.5718 ± 0.0131 ~ !


## Analyze the results

In [19]:
# Let's have a look at the final results
atom.results

,f1_train,f1_test,time_fit,f1_bootstrap,time_bootstrap,time
ET,0.8566,0.5615,0.273168,0.565538,0.902319,1.175487
RF,1.0000,0.5714,0.261275,0.571824,1.011731,1.273006


In [20]:
# Visualize the bootstrap results
atom.plot_results(title="RF vs ET performance")

In [21]:
# Print the results of some common metrics
atom.evaluate()

,accuracy,ap,ba,f1,jaccard,mcc,precision,recall,auc
ET,0.846600,0.673300,0.703600,0.561500,0.390300,0.497800,0.746900,0.449800,0.851100
RF,0.845000,0.674700,0.714600,0.576500,0.405000,0.499800,0.714300,0.483300,0.847100


In [22]:
# The winner attribute calls the best model (atom.winner == atom.rf)
print(f"The winner is the {atom.winner.name} model!!")

The winner is the RF model!!


In [23]:
# Visualize the distribution of predicted probabilities
atom.winner.plot_probabilities()

In [24]:
# Compare how different metrics perform for different thresholds
atom.winner.plot_threshold(metric=["f1", "accuracy", "ap"], steps=50)